In [ ]:
import numpy as np
import pickle
from scripts import (
    train_test_split,
    StandardScaler,
    accuracy,
    confusion_matrix,
    recall_per_class,
    balanced_accuracy,
    KernelPerceptron,
    SVM,
    rbf_kernel,
)
import pandas as pd

In [66]:
np.random.seed(0)

In [67]:
# Chemin vers le fichier contenant les données d'entraînement
path_to_data = 'ift-3395-6390-kaggle-2-competition-fall-2025/train_data.pkl'

# Chargement des données à partir du fichier pickle
with open(path_to_data, "rb") as f:
    train_data = pickle.load(f)

# Extraction des images et conversion en float32
X_imgs = train_data["images"].astype(np.float32)

# Extraction des labels et mise en forme en vecteur 1D
y = train_data["labels"].reshape(-1)

# Séparation des données en ensembles d'entraînement et de validation
X_train_imgs, X_val_imgs, y_train, y_val = train_test_split(
    X_imgs, y, test_size=0.2, random_state=0
)

# Calcul des statistiques uniquement sur l'ensemble d'entraînement
train_min = X_train_imgs.min()
train_max = X_train_imgs.max()
train_mean = X_train_imgs.mean()
train_std = X_train_imgs.std()

# Normalisation min-max suivie d'une standardisation 
X_train_imgs = (X_train_imgs - train_min) / (train_max - train_min + 1e-6)
X_train_imgs = (X_train_imgs - train_mean) / (train_std + 1e-6)

# Application des mêmes transformations à l'ensemble de validation
X_val_imgs = (X_val_imgs - train_min) / (train_max - train_min + 1e-6)
X_val_imgs = (X_val_imgs - train_mean) / (train_std + 1e-6)


In [68]:
def extract_simple_stats(img):
    # Conversion de l'image RGB en greyscale
    gray = img.mean(axis=2)

    # Calcul de statistiquessur l'image: moyenne, écart-type, minimum et maximum
    return np.array(
        [gray.mean(), gray.std(), gray.min(), gray.max()],
        dtype=np.float32
    )


In [69]:
def radial_profile(img):
    # Récupération de la hauteur et de la largeur de l'image
    h, w = img.shape

    # Création des grilles de coordonnées pour chaque pixel
    y, x = np.ogrid[:h, :w]

    # Calcul de la distance de chaque pixel par rapport au centre de l'image
    r = np.sqrt((x - w // 2) ** 2 + (y - h // 2) ** 2).astype(int)

    # Calcul du profil radial 
    profile = (
        np.bincount(r.ravel(), img.ravel()) /
        np.bincount(r.ravel())
    )

    # Application d'une transformation logarithmique
    return np.log(profile + 1e-6)


In [70]:
def fft_features(images):
    # Conversion des images RGB en greyscale
    gray = images.mean(axis=3)

    # calcul du Fast Fourier Trasnform 
    F = np.fft.fft2(gray, axes=(1, 2))

    # calcul du module du spectre de Fourier
    return np.abs(np.fft.fftshift(F, axes=(1, 2)))


In [71]:
def extract_channel_stats(img):
    # Liste pour stocker les caractéristiques extraites
    features = []

    # Itération sur les canaux de couleur (R, G, B)
    for c in range(img.shape[2]):
        # Extraction du canal courant
        channel = img[:, :, c]

        # Calcul de statistiques sur ce canal :
        features.extend([
            channel.mean(),
            channel.std(),
            channel.min(),
            channel.max()
        ])

    # Conversion de la liste en tableau NumPy
    return np.array(features, dtype=np.float32)


In [72]:
def simple_augment(images, labels):
    # Augmentation par flip horizontal des images
    flips = images[:, :, ::-1, :]

    # Augmentation par ajout de bruit gaussien léger
    noise = images + 0.01 * np.random.randn(*images.shape)

    # Concaténation des images originales et augmentées
    aug_imgs = np.concatenate([images, flips, noise], axis=0)

    # Duplication des labels correspondants aux images augmentées
    aug_labels = np.concatenate([labels, labels, labels])

    return aug_imgs, aug_labels


In [73]:
X_train_imgs, y_train = simple_augment(X_train_imgs, y_train)

In [74]:

# Extraction des caractéristiques FFT sur les images d'entraînement
fft_mag_train = fft_features(X_train_imgs)

# Calcul du profil radial pour chaque image 
X_fft_train = np.array(
    [radial_profile(img) for img in fft_mag_train],
    dtype=np.float32
)

# Extraction de statistiques simples 
X_stats_train = np.array(
    [extract_simple_stats(img) for img in X_train_imgs],
    dtype=np.float32
)

# Extraction de statistiques par canal de couleur 
X_color_train = np.array(
    [extract_channel_stats(img) for img in X_train_imgs],
    dtype=np.float32
)

# Extraction des caractéristiques FFT sur les images de validation
fft_mag_val = fft_features(X_val_imgs)

# Calcul du profil radial pour chaque image 
X_fft_val = np.array(
    [radial_profile(img) for img in fft_mag_val],
    dtype=np.float32
)

# Extraction de statistiques simples (
X_stats_val = np.array(
    [extract_simple_stats(img) for img in X_val_imgs],
    dtype=np.float32
)

# Extraction de statistiques par canal de couleur 
X_color_val = np.array(
    [extract_channel_stats(img) for img in X_val_imgs],
    dtype=np.float32
)


In [75]:
# Concaténation des différentes caractéristiques 
X_train = np.hstack([X_fft_train, X_stats_train, X_color_train])
X_val = np.hstack([X_fft_val, X_stats_val, X_color_val])


In [76]:
# Normalisation 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [77]:
# Calcul du nombre d'exemples par classe dans l'ensemble de validation
class_counts = np.bincount(y_val)

# Calcul de poids des classes
class_weights = (1.0 / class_counts)

# Normalisation des poids 
class_weights /= class_weights.sum()

# Attribution d'un poids à chaque échantillon de l'ensemble d'entraînement
sample_weights = class_weights[y_train]


In [ ]:
# Initialisation du perceptron à noyau avec un noyau RBF
model = KernelPerceptron(
    kernel_fn=rbf_kernel,     
    n_classes=5,        
    sigma=1.0,              
    learning_rate=1.0,          # Taux d'apprentissage
    sample_weights=sample_weights,  # Poids des échantillons pour gérer le déséquilibre
    lam=0.0                         # Terme de régularisation
)

# Entraînement du modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train, max_epochs=50)


In [85]:
y_pred_val = model.predict(X_val)
acc = (y_pred_val == y_val).mean()
print("Test accuracy =", acc)
cm = confusion_matrix(y_val, y_pred_val)
bal_acc = balanced_accuracy(y_val, y_pred_val)
rec = recall_per_class(cm)
print("Balanced acc :", bal_acc)
print("Recall par classe :", rec)
print("Recall moyen :", rec.mean())
print(cm)



Test accuracy = 0.4583333333333333
Balanced acc : 0.2957273553703023
Recall par classe : [0.77319588 0.15384615 0.24390244 0.23076923 0.07692308]
Recall moyen : 0.2957273553703023
[[75 10  5  6  1]
 [ 7  4  4  8  3]
 [16  4 10 10  1]
 [11  5 12  9  2]
 [ 2  2  3  5  1]]


In [80]:
model_pkg = {
'model': model,
'scaler': scaler,
'train_min': train_min,
'train_max':train_max,
'train_mean': train_mean,
'train_std': train_std
}

pickle.dump(model_pkg, open("model_perceptron.pkl","wb"))

In [ ]:
# Chargement du modèle entraîné et des paramètres de prétraitement sauvegardés
model_pkg_pred = pickle.load(open("model_perceptron.pkl", "rb"))

# Récupération du modèle 
model = model_pkg_pred['model']
scaler = model_pkg_pred['scaler']
train_min = model_pkg_pred['train_min']
train_max = model_pkg_pred['train_max']
train_mean = model_pkg_pred['train_mean']
train_std = model_pkg_pred['train_std']


# Chargement des données de test

with open("ift-3395-6390-kaggle-2-competition-fall-2025/test_data.pkl", "rb") as f:
    test_data = pickle.load(f)

# Extraction des images de test
X_test_imgs = test_data["images"].astype(np.float32)

# Application de la même normalisation que pour l'ensemble d'entraînement
X_test_imgs = (X_test_imgs - train_min) / (train_max - train_min + 1e-6)
X_test_imgs = (X_test_imgs - train_mean) / (train_std + 1e-6)

# Extraction des caractéristiques fréquentielles pour l'ensemble de test
fft_mag_test = fft_features(X_test_imgs)

# Calcul du profil radial
X_fft_test = np.array(
    [radial_profile(img) for img in fft_mag_test],
    dtype=np.float32
)

# Extraction de statistiques simples 
X_stats_test = np.array(
    [extract_simple_stats(img) for img in X_test_imgs],
    dtype=np.float32
)

# Extraction de statistiques par canal de couleur 
X_color_test = np.array(
    [extract_channel_stats(img) for img in X_test_imgs],
    dtype=np.float32
)

# Concaténation de toutes les caractéristiques extraites
X_test = np.hstack([X_fft_test, X_stats_test, X_color_test])

# Application de la standardisation 
X_test = scaler.transform(X_test)

# Prédiction des classes sur l'ensemble de test
y_pred = model.predict(X_test).astype(int)

# Génération du fichier CSV pour la soumission Kaggle
df = pd.DataFrame({
    "ID": np.arange(1, len(y_pred) + 1),
    "Label": y_pred
})

# Sauvegarde du fichier CSV
df.to_csv("ift3395_YAPS_MCS_V101.csv", index=False)





Fichier 'submission.csv' généré !
    Label      
     self other
4     2.0   0.0
6     2.0   4.0
9     1.0   0.0
11    4.0   0.0
12    2.0   0.0
..    ...   ...
394   1.0   0.0
395   3.0   0.0
396   1.0   0.0
397   3.0   1.0
399   3.0   0.0

[196 rows x 2 columns]
Nombre de différences : 196
